In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os, time, json
import pandas as pd
from openai import OpenAI
from anthropic import Anthropic
from tqdm.auto import tqdm
import spacy

import sys

sys.path.append("../../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

# client = OpenAI(
#     api_key=os.getenv("OPENAI_KEY"),
# )
client = Anthropic(
    api_key=os.getenv("CLAUDE_KEY"),
)

MODEL_NAME = "claude-3-5-sonnet-20240620"

2024-09-05 17:05:05 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-09-05 17:05:05 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-09-05 17:05:05 __main__ INFO     transformers.__version__='4.44.2'
2024-09-05 17:05:05 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2024-09-05 17:05:05 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'


In [41]:
# # movie,actor,character
# prompt = """
# Give me a list of movies where an actor potrayed a famous person. It should be in csv format. Give me as many examples as you can.

# movie,actor,character
# A Beautiful Mind,Russell Crowe,John Nash
# Gandhi,Ben Kingsley,Mahatma Gandhi
# """

In [42]:
# # superhero,similar powers, characteristics

# prompt = """
# Give me a list of superpowered characters with similar powers and traits. The character can be from any universe (Marvel, DC, different anime etc). It should be in csv format. Give me as many examples as you can.

# connection,character1,character2
# kryptonian,Superman,Supergirl
# speedster,Flash,Quicksilver
# billionaire,Batman,Ironman
# blind, Daredevil, Toph Beifong
# """

In [43]:
# # architect,famous buildings

# prompt = """
# Give me a list of 2 famous buildings designed by the same architect or architectural firm. It should be in csv format. Give me as many examples as you can (atleast 20).
# It should be in csv format. Give me as many examples as you can.

# architect,building1,building2
# Frank Lloyd Wright,Fallingwater,Guggenheim Museum
# Zaha Hadid,Heydar Aliyev Center,Guangzhou Opera House
# """

In [44]:
# # sports, famous players

# prompt = """
# Give me a list of 2 famous players who played the same sport. It should be in csv format. Give me as many examples as you can (atleast 50).

# sport,player1,player2
# basketball,Michael Jordan,LeBron James
# tennis,Roger Federer,Rafael Nadal
# Cricket,Michael Vaughan,Sourav Ganguly
# """

In [45]:
# None

prompt = """
Give me a list of 2 famous entities such that there is no obvious connection between them. The entities can be anything (people, places, things, events etc). 
It should be in csv format. Give me as many examples as you can (atleast 50).

connection,entity1,entity2
None,Michael Jordan,Slovenia
None,Mount Everest,Jon Bon Jovi
"""

In [46]:
# response = client.chat.completions.create(
#     model=MODEL_NAME,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0,
#     max_tokens=4000,
# )
# print(response.choices[0].message.content)

# response = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=1000,
#     temperature=0,
#     system="You are a helpful assistant.",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": prompt,
#                 }
#             ]
#         }
#     ]
# )
# print(response.content[0].text)

In [47]:
from src.functional import ask_claude

print(ask_claude(prompt))

2024-09-05 17:16:23 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2024-09-05 17:16:23 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'
2024-09-05 17:16:23 anthropic._base_client DEBUG    Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': 600, 'files': None, 'json_data': {'max_tokens': 4000, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': '\nGive me a list of 2 famous entities such that there is no obvious connection between them. The entities can be anything (people, places, things, events etc). \nIt should be in csv format. Give me as many examples as you can (atleast 50).\n\nconnection,entity1,entity2\nNone,Michael Jordan,Slovenia\nNone,Mount Everest,Jon Bon Jovi\n'}]}], 'model': 'claude-3-5-sonnet-20240620', 'system': 'You are a helpful assistant.', 'temperature': 0}}
2024-09-05 17:16:23 anthropic._base_client DEBUG    Send

IOStream.flush timed out


2024-09-05 17:16:42 httpcore.http11 DEBUG    receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 05 Sep 2024 21:16:42 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'anthropic-ratelimit-requests-limit', b'1000'), (b'anthropic-ratelimit-requests-remaining', b'999'), (b'anthropic-ratelimit-requests-reset', b'2024-09-05T21:17:10Z'), (b'anthropic-ratelimit-tokens-limit', b'80000'), (b'anthropic-ratelimit-tokens-remaining', b'80000'), (b'anthropic-ratelimit-tokens-reset', b'2024-09-05T21:16:42Z'), (b'request-id', b'req_01J74aLU2DoqUFGpRLtiKEWt'), (b'x-cloud-trace-context', b'7b3981d288de8550dba97c69b4bdcfb1'), (b'via', b'1.1 google'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Robots-Tag', b'none'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8be93db7d9049020-BOS'), (b'Content-Encoding', b'gzip')])
2024-09-05 17:16:42 httpx INFO     HTTP Request: POST https://api.anthropic.com/v1/messages 